In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm
from datetime import datetime
from pandas_datareader.data import DataReader
from statsmodels.tsa.regime_switching import markov_switching
import hmms

from hmmlearn import hmm


# Get Data 
For 10 year treasury bonds, FTSE and 30 year treasurt bonds

In [2]:
from datetime import datetime
start_date = datetime(1900, 1, 1)
end_date = datetime.now()
ftse_all_world_data = yf.download('VT', period='max', interval='1mo')
treasury_bond_15yr_data = yf.download('TLH',period='max', interval='1mo')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
print(ftse_all_world_data)
print(treasury_bond_15yr_data)

                  Open        High        Low       Close   Adj Close  \
Date                                                                    
2008-07-01   49.130001   49.689999  46.340000   48.209999   34.741230   
2008-08-01   48.189999   50.020000  46.139999   47.320000   34.099865   
2008-09-01   47.320000   47.689999  35.849998   43.009998   30.993986   
2008-10-01   41.590000   42.830002  29.000000   33.790001   24.349846   
2008-11-01   34.650002   35.990002  26.600000   31.490000   22.692406   
...                ...         ...        ...         ...         ...   
2023-07-01   97.059998  100.730003  94.970001  100.589996  100.168800   
2023-08-01  100.080002  100.209999  94.230003   97.730003   97.320778   
2023-09-01   98.529999   98.639999  92.129997   93.180000   92.789825   
2023-10-01   92.989998   94.709999  88.739998   90.459999   90.459999   
2023-11-01   90.650002   98.360001  90.519997   98.330002   98.330002   

              Volume  
Date                  
2008

# Prepare Data

In [4]:
frame_1 = pd.DataFrame()
frame_1['Stocks'] = (ftse_all_world_data['Close'])
frame_1['Bonds'] = (treasury_bond_15yr_data['Close'])
frame = np.log(frame_1)
df = frame.diff()[1:]
print(frame,df)


              Stocks     Bonds
Date                          
2008-07-01  3.875566  4.655483
2008-08-01  3.856933  4.670302
2008-09-01  3.761433  4.669271
2008-10-01  3.520165  4.627030
2008-11-01  3.449670  4.734530
...              ...       ...
2023-07-01  4.611053  4.685274
2023-08-01  4.582209  4.659658
2023-09-01  4.534533  4.589752
2023-10-01  4.504908  4.545208
2023-11-01  4.588329  4.601162

[185 rows x 2 columns]               Stocks     Bonds
Date                          
2008-08-01 -0.018633  0.014819
2008-09-01 -0.095500 -0.001031
2008-10-01 -0.241268 -0.042242
2008-11-01 -0.070495  0.107501
2008-12-01  0.046231  0.066289
...              ...       ...
2023-07-01  0.036548 -0.022182
2023-08-01 -0.028844 -0.025616
2023-09-01 -0.047675 -0.069906
2023-10-01 -0.029625 -0.044544
2023-11-01  0.083421  0.055954

[184 rows x 2 columns]


# MS-VAR

In [5]:
# Assuming df is your DataFrame and it contains one-dimensional observations
data = df.values


In [6]:
# Number of states
num_states = 3

# Create a Gaussian HMM
model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag")

# Fit the model to your data
model.fit(data)


GaussianHMM(n_components=3)

In [7]:
print("Transition matrix:")
print(model.transmat_)
print("Means and variances of each state:")
for i in range(num_states):
    print(f"State {i}: mean = {model.means_[i]}, variance = {model.covars_[i]}")
hidden_states = model.predict(data)  
print("Hidden states:")
print(hidden_states)
log_likelihood = model.score(data)
print("Log likelihood of the model:")
print(log_likelihood)
n_samples = 10
samples, _ = model.sample(n_samples)
print(f"Generated samples ({n_samples}):")
print(samples)


Transition matrix:
[[0.9528126  0.04403629 0.00315111]
 [0.4214324  0.04449224 0.53407536]
 [0.00383251 0.47159178 0.52457571]]
Means and variances of each state:
State 0: mean = [ 0.00965582 -0.00098446], variance = [[0.00134915 0.        ]
 [0.         0.0005697 ]]
State 1: mean = [0.01868886 0.00985897], variance = [[0.00541063 0.        ]
 [0.         0.0023549 ]]
State 2: mean = [-0.04751043 -0.0037187 ], variance = [[0.00729128 0.        ]
 [0.         0.0030212 ]]
Hidden states:
[2 2 2 2 1 2 2 1 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 2 1 2 2 2 1 2 1 0 0 0 0 0 0 0 1 2 1]
Log likelihood of the model:
708.3018523943575
Generated samples (10):
[[-0.02677531 -0.11130855]
 [ 0.08900734  0.05473725]
 [ 0.10321